In [ ]:
cd ..

In [ ]:
import pandas as pd
import numpy as np
from pyquaternion import Quaternion
import math
from typing import List

In [ ]:
cameras = pd.read_csv("data/cameras.csv")
cameras

In [ ]:
# all cameras in experiment are front-facing
front_camera_translation = [1.72200568478, 0.00475453292289, 1.49491291905]

In [ ]:
from apperception.database import database

query = "SELECT filename, getX(egoTranslation), getY(egoTranslation), getZ(egoTranslation) FROM Cameras;"

result = database._execute_query(query=query)

In [ ]:
# convert to mapping from filename -> (x, y, z) = egoRotation
ego_translations = {}
for camera in result:
    filename, x, y, z = camera
    ego_translations[filename] = [x, y, z]

In [ ]:
def get_ego_translation(camera):
    return ego_translations[camera.filename]

def get_ego_rotation(camera):
    return [float(x) for x in camera.egorotation.strip("{").strip("}").split(",")]

def normalizeAngle(angle) -> float:
    while angle > math.pi:
        angle -= math.tau
    while angle < -math.pi:
        angle += math.tau
    assert -math.pi <= angle <= math.pi
    return angle

def get_heading_from_north(rotation):
    yaw = rotation.yaw_pitch_roll[0]
    return normalizeAngle(yaw - (math.pi / 2))

def get_camera_position(
    camera_translation: List[float],
    ego_translation: List[float],
    ego_rotation: List[float]
) -> np.ndarray:
    rotated_offset = Quaternion(ego_rotation) \
        .rotate(np.array(camera_translation))
    cam_position = np.array(ego_translation) + rotated_offset
    return "Point(" + " ".join(map(str, cam_position)) + ")"

In [ ]:
# cameras[cameras["filename"] == 'samples/CAM_FRONT/n008-2018-09-18-13-10-39-0400__CAM_FRONT__1537290952162404.jpg']["cameraTranslationAbs"][94]

In [ ]:
cameras["cameraTranslationAbs"] = cameras.apply(lambda x: get_camera_position(front_camera_translation, get_ego_translation(x), get_ego_rotation(x)), axis=1)

In [ ]:
cameras.to_csv("data/cameras.csv", index=False)